<a href="https://colab.research.google.com/github/ValentinaEmili/Sign_language/blob/main/ASL_recognition_100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The glossary is made of 100 different words but the instances for each word are not the same as the ones in WLASL_v0.3 file. Indeed, some links were broken and the correspective instances have been removed. Every word has at least one instance.

In [1]:
# mount google drive on colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import cv2
from google.colab.patches import cv2_imshow
from tqdm import tqdm
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_sequence, pad_packed_sequence
from torch.nn import LSTM
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Dataset
import shutil

In [3]:
class SignLanguageDataset(Dataset):
  def __init__(self, image_dir, label_map, vis):
    self.image_dir = image_dir
    self.label_map = label_map
    self.files = sorted(os.listdir(image_dir))
    self.vis = vis

  def __len__(self):
    return len(self.files)

  def __getitem__(self, idx):
    file_name = self.files[idx]
    np_array = np.load(os.path.join(self.image_dir, file_name))
    if self.vis:
      if np_array.size == 0 or len(np_array.shape) != 2 or np_array.shape[1] != 258:
        print(f"Warning: Empty or invalid shape for file: {file_name}")
        np_array = np.zeros((1, 258), dtype=np.float32)
    else:
      if np_array.size == 0 or len(np_array.shape) != 2 or np_array.shape[1] != 225:
        print(f"Warning: Empty or invalid shape for file: {file_name}")
        np_array = np.zeros((1, 225), dtype=np.float32)

    label, _ = file_name.split("_")
    label = self.label_map[label]

    return torch.tensor(np_array, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

# Add zero-padding to get sequences of the same length for each batch
def collate_fn(batch):
  sequences, labels = zip(*batch)
  lengths = [len(seq) for seq in sequences]
  padded_sequences = pad_sequence(sequences, batch_first=True)

  # pack the padded sequence
  packed_sequences = pack_padded_sequence(padded_sequences, lengths, batch_first=True, enforce_sorted=False)
  return packed_sequences, torch.tensor(labels)

code inspired to:

https://medium.com/@eugenesh4work/attention-mechanism-for-lstm-used-in-a-sequence-to-sequence-task-be1d54919876

LSTM with Attention Mechanism

In [14]:
class Attention(nn.Module):
  def __init__(self, hidden_size):
    super(Attention, self).__init__()
    self.hidden_size = hidden_size

  def forward(self, lstm_output, final_hidden_state):
    # lstm_output: (batch_size, seq_len, hidden_size * 2)
    # final hidden state: (batch_size, hidden_size * 2)
    scores = torch.bmm(lstm_output, final_hidden_state.unsqueeze(2)).squeeze(2) # (batch_size, seq_len)
    attention_weights = F.softmax(scores, dim=1) # (batch_size, seq_len)
    context_vector = torch.bmm(attention_weights.unsqueeze(1), lstm_output).squeeze(1)  # (batch_size, hidden_dim * 2)
    return context_vector, attention_weights

class SignLanguageLSTM(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes, dropout_rate=0.5):
    super(SignLanguageLSTM, self).__init__()

    # input regularization
    self.input_bn = nn.BatchNorm1d(input_size)
    self.input_dropout = nn.Dropout(0.3)
    #self.input_dropout = nn.Dropout(0.2)

    # single bidirectional LSTM layer
    self.lstm = nn.LSTM(
        input_size=input_size,
        hidden_size=hidden_size,
        batch_first=True,
        num_layers=2,
        dropout=dropout_rate,
        bidirectional=True)

    # attention mechanism
    self.attention = Attention(hidden_size)

    # fully connected layers
    self.fc1 = nn.Linear(hidden_size * 2, hidden_size)
    self.fc2 = nn.Linear(hidden_size, num_classes)

    self.dropout = nn.Dropout(dropout_rate)

  def forward(self, packed_input):
    # unpack input for batch normalization
    padded_input, lengths = pad_packed_sequence(packed_input, batch_first=True)

    # apply input normalization and dropout
    padded_input = padded_input.transpose(1, 2)
    padded_input = self.input_bn(padded_input)
    padded_input = padded_input.transpose(1, 2)
    padded_input = self.input_dropout(padded_input)

    # re-pack input
    packed_input = pack_padded_sequence(padded_input, lengths, batch_first=True, enforce_sorted=False)

    # LSTM
    packed_output, (hn, cn) = self.lstm(packed_input) # (batch_size, seq_len, hidden_size * 2), (2, batch_size, hidden_size), (2, batch_size, hidden_size)

    output_forward = hn[0, :, :] # last hidden state for forward direction (batch_size, hidden_size)
    output_backward = hn[1, :, :] # last hidden state for backward direction (batch_size, hidden_size)
    final_hidden_state = torch.cat((output_forward, output_backward), dim=1) # (batch_size, hidden_size * 2)

    lstm_output, _ = pad_packed_sequence(packed_output, batch_first=True) # (batch_size, seq_len, hidden_size * 2)
    context_vector, _ = self.attention(lstm_output, final_hidden_state) # (batch_size, hidden_size * 2)

    output = F.relu(self.fc1(context_vector)) # (batch_size, hidden_size)
    output = self.fc2(self.dropout(output)) # (hidden_size, num_classes)

    return output

GRU with Attention Mechanism

In [ ]:
class Attention(nn.Module):
  def __init__(self, hidden_size):
    super(Attention, self).__init__()
    self.hidden_size = hidden_size

  def forward(self, lstm_output, final_hidden_state):
    # lstm_output: (batch_size, seq_len, hidden_size * 2)
    # final hidden state: (batch_size, hidden_size * 2)
    scores = torch.bmm(lstm_output, final_hidden_state.unsqueeze(2)).squeeze(2) # (batch_size, seq_len)
    attention_weights = F.softmax(scores, dim=1) # (batch_size, seq_len)
    context_vector = torch.bmm(attention_weights.unsqueeze(1), lstm_output).squeeze(1)  # (batch_size, hidden_dim * 2)
    return context_vector, attention_weights

class SignLanguageGRU(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes, dropout_rate=0.5):
    super(SignLanguageGRU, self).__init__()

    # input regularization
    self.input_bn = nn.BatchNorm1d(input_size)
    self.input_dropout = nn.Dropout(0.3)

    # single bidirectional GRU layer
    self.gru = nn.GRU(
        input_size=input_size,
        hidden_size=hidden_size,
        batch_first=True,
        dropout=dropout_rate,
        bidirectional=True)

    # attention mechanism
    self.attention = Attention(hidden_size)

    # fully connected layers
    self.fc1 = nn.Linear(hidden_size * 2, hidden_size)
    self.fc2 = nn.Linear(hidden_size, num_classes)

    self.dropout = nn.Dropout(dropout_rate)

  def forward(self, packed_input):
    # unpack input for batch normalization
    padded_input, lengths = pad_packed_sequence(packed_input, batch_first=True)

    # apply input normalization and dropout
    padded_input = padded_input.transpose(1, 2)
    padded_input = self.input_bn(padded_input)
    padded_input = padded_input.transpose(1, 2)
    padded_input = self.input_dropout(padded_input)

    # re-pack input
    packed_input = pack_padded_sequence(padded_input, lengths, batch_first=True, enforce_sorted=False)

    # GRU
    packed_output, hn = self.gru(packed_input)

    output_forward = hn[0, :, :]
    output_backward = hn[1, :, :]
    final_hidden_state = torch.cat((output_forward, output_backward), dim=1)

    gru_output, _ = pad_packed_sequence(packed_output, batch_first=True)
    context_vector, _ = self.attention(gru_output, final_hidden_state)

    output = F.relu(self.fc1(context_vector))
    output = self.fc2(self.dropout(output))

    return output

In [5]:
def train(model, num_epochs, train_loader, val_loader, optimizer, criterion, scheduler, device, training_history, best_accuracy):
  for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs} [Train]'):
      inputs, labels = inputs.to(device), labels.to(device)

      optimizer.zero_grad()
      outputs = model(inputs)
      loss = criterion(outputs, labels)
      loss.backward()

      torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

      optimizer.step()

      running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_train_loss:.4f}')

    # evaluation phase
    model.eval()
    val_loss, correct, total = 0, 0, 0

    with torch.no_grad():
      for inputs, labels in tqdm(val_loader, desc=f'Epoch {epoch + 1}/{num_epochs} [Valid]'):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        val_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    accuracy = correct / total
    avg_val_loss = val_loss / len(val_loader)

    # update learning rate based on validation accuracy
    scheduler.step(accuracy)

    print(f'Validation Accuracy: {accuracy * 100:.2f}%')
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {avg_val_loss:.4f}')

    # store training history
    training_history.append({
      'epoch': epoch + 1,
      'train_loss': avg_train_loss,
      'val_loss': avg_val_loss,
      'acc': round(accuracy * 100, 2),  # store as percentage
      'lr': optimizer.param_groups[0]['lr']
      })

    # save best model
    if accuracy > best_accuracy:
      best_accuracy = accuracy
      #torch.save({
      #  'epoch': epoch,
      #  'model_state_dict': model.state_dict(),
      #  'optimizer_state_dict': optimizer.state_dict(),
      #  'accuracy': accuracy, # saved as decimal
      #  'val_loss': avg_val_loss,
      #}, '/content/drive/MyDrive/NLP/saved_models/best_model_100.pth')
      print(f'Saved new best model with accuracy: {best_accuracy * 100:.2f}%\n-----')

In [6]:
def evaluate(model, test_loader, device):
  model.eval()
  correct, total = 0, 0
  with torch.no_grad():
    for inputs, labels in test_loader:
      inputs, labels = inputs.to(device), labels.to(device)
      outputs = model(inputs)
      _, predicted = torch.max(outputs, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
  test_accuracy = correct / total  # Test accuracy
  print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

In [17]:
def main():
  js_100 = pd.read_json("/content/drive/MyDrive/NLP/WLASL100.json")
  folder = "/content/drive/MyDrive/NLP/dataset/subset_100/"
  original_folder = "/content/drive/MyDrive/NLP/dataset/"
  vis = False

  training_folder = folder + "train/"
  validation_folder = folder + "val/"
  test_folder = folder + "test/"

  training_video = training_folder + "video/"
  validation_video = validation_folder + "video/"
  test_video = test_folder + "video/"

  if vis:
    training_images = training_folder + "images/"
    validation_images = validation_folder + "images/"
    test_images = test_folder + "images/"
  else:
    training_images = training_folder + "images_no_vis/"
    validation_images = validation_folder + "images_no_vis/"
    test_images = test_folder + "images_no_vis/"

  gloss = set()
  for image in os.listdir(training_images):
    word, _ = image.split("_")
    gloss.add(word)
  label_map = {label: num for num, label in enumerate(gloss)}

  train_dataset = SignLanguageDataset(training_images, label_map, vis)
  val_dataset = SignLanguageDataset(validation_images, label_map, vis)
  test_dataset = SignLanguageDataset(test_images, label_map, vis)

  batch_size = 16
  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
  val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
  test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

  def get_training_config():
    return {
      'hidden_size': 512,
      'learning_rate': 1e-4,
      'num_epochs': 200,
      'weight_decay': 1e-4,
      'dropout_rate': 0.2,
      'scheduler_params': {
        'factor': 0.5,
        'min_lr': 1e-6
      },
    }

  best_accuracy = 0.0
  training_history = []
  config = get_training_config()
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  num_epochs = config['num_epochs']

  input_size = 258 if vis else 225

  model = SignLanguageLSTM(
  #model = SignLanguageGRU(
      input_size=input_size,
      hidden_size = config['hidden_size'],
      num_classes = len(label_map),
      dropout_rate=config['dropout_rate']).to(device)

  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(
      model.parameters(),
      lr=config['learning_rate'],
      weight_decay=config['weight_decay'])

  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
      optimizer,
      mode='max',
      factor=config['scheduler_params']['factor'],
      min_lr=config['scheduler_params']['min_lr'],
      patience=5)

  train(model, num_epochs, train_loader, val_loader, optimizer, criterion, scheduler, device, training_history, best_accuracy)
  evaluate(model, test_loader, device)

if __name__ == "__main__":
  main()

Epoch 1/200 [Train]: 100%|██████████| 58/58 [00:08<00:00,  6.61it/s]


Epoch [1/200], Loss: 4.5638


Epoch 1/200 [Valid]: 100%|██████████| 14/14 [00:00<00:00, 14.39it/s]


Validation Accuracy: 2.84%
Epoch [1/200], Validation Loss: 4.4453
Saved new best model with accuracy: 2.84%
-----


Epoch 2/200 [Train]: 100%|██████████| 58/58 [00:06<00:00,  8.75it/s]


...

Validation Accuracy: 45.97%
Epoch [199/200], Validation Loss: 2.6488


Epoch 200/200 [Train]: 100%|██████████| 58/58 [00:05<00:00, 10.12it/s]


Epoch [200/200], Loss: 0.3717


Epoch 200/200 [Valid]: 100%|██████████| 14/14 [00:00<00:00, 15.31it/s]


Validation Accuracy: 43.13%
Epoch [200/200], Validation Loss: 2.6624
Test Accuracy: 38.10%


Best params with visibility information (LSTM)

In [ ]:
def get_training_config():
  return {
    'hidden_size': 512,
    'learning_rate': 1e-4,
    'num_epochs': 100,
    'weight_decay': 1e-4,
    'dropout_rate': 0.1,
    'scheduler_params': {
      'factor': 0.5,
      'min_lr': 1e-6
    },
  }

  # Validation Accuracy: 48.34%
  # Test Accuracy: 39.15%

def get_training_config():
  return {
    'hidden_size': 512,
    'learning_rate': 1e-4,
    'num_epochs': 100,
    'weight_decay': 1e-4,
    'dropout_rate': 0.2,
    'scheduler_params': {
      'factor': 0.5,
      'min_lr': 1e-6
    },
  }

  # Validation Accuracy: 49.76%
  # Test Accuracy: 40.21%

def get_training_config():
  return {
    'hidden_size': 512,
    'learning_rate': 1e-4,
    'num_epochs': 100,
    'weight_decay': 1e-5,
    'dropout_rate': 0.2,
    'scheduler_params': {
      'factor': 0.5,
      'min_lr': 1e-6
    },
  }
  # Validation Accuracy: 47.87%
  # Test Accuracy: 38.10%

def get_training_config():
  return {
    'hidden_size': 512,
    'learning_rate': 1e-4,
    'num_epochs': 100,
    'weight_decay': 1e-5,
    'dropout_rate': 0.3,
    'scheduler_params': {
      'factor': 0.5,
      'min_lr': 1e-6
    },
  }
  # Validation Accuracy: 47.39%
  # Test Accuracy: 37.04%

def get_training_config():
  return {
    'hidden_size': 512,
    'learning_rate': 1e-4,
    'num_epochs': 100,
    'weight_decay': 1e-4,
    'dropout_rate': 0.3,
    'scheduler_params': {
      'factor': 0.5,
      'min_lr': 1e-6
    },
  }
  # Validation Accuracy: 47.87%
  # Test Accuracy: 37.04%

def get_training_config():
  return {
    'hidden_size': 512,
    'learning_rate': 1e-3,
    'num_epochs': 100,
    'weight_decay': 1e-4,
    'dropout_rate': 0.2,
    'scheduler_params': {
      'factor': 0.5,
      'min_lr': 1e-6
    },
  }
  # Validation Accuracy: 47.39%
  # Test Accuracy: 38.62%

def get_training_config():
  return {
    'hidden_size': 512,
    'learning_rate': 1e-3,
    'num_epochs': 200,
    'weight_decay': 1e-4,
    'dropout_rate': 0.2,
    'scheduler_params': {
      'factor': 0.5,
      'min_lr': 1e-6
    },
  }
  # Validation Accuracy: 45.97%
  # Test Accuracy: 33.86%

Best params without visibility information (LSTM)

In [ ]:
def get_training_config():
  return {
    'hidden_size': 512,
    'learning_rate': 1e-4,
    'num_epochs': 100,
    'weight_decay': 1e-4,
    'dropout_rate': 0.2,
    'scheduler_params': {
      'factor': 0.5,
      'min_lr': 1e-6
    },
  }

  # Validation Accuracy: 50.24%
  # Test Accuracy: 38.10%

def get_training_config():
  return {
    'hidden_size': 512,
    'learning_rate': 1e-4,
    'num_epochs': 200,
    'weight_decay': 1e-4,
    'dropout_rate': 0.3,
    'scheduler_params': {
      'factor': 0.5,
      'min_lr': 1e-6
    },
  }

  # Validation Accuracy: 48.34%
  # Test Accuracy: 40.74%

def get_training_config():
  return {
    'hidden_size': 512,
    'learning_rate': 1e-4,
    'num_epochs': 200,
    'weight_decay': 1e-5,
    'dropout_rate': 0.3,
    'scheduler_params': {
      'factor': 0.5,
      'min_lr': 1e-6
    },
  }
  # Validation Accuracy: 47.87%
  # Test Accuracy: 35.45%

def get_training_config():
  return {
    'hidden_size': 512,
    'learning_rate': 1e-4,
    'num_epochs': 200,
    'weight_decay': 1e-4,
    'dropout_rate': 0.2,
    'scheduler_params': {
      'factor': 0.5,
      'min_lr': 1e-6
    },
  }
  # Validation Accuracy: 51.18%
  # Test Accuracy: 40.74%

def get_training_config():
  return {
    'hidden_size': 512,
    'learning_rate': 1e-3,
    'num_epochs': 200,
    'weight_decay': 1e-4,
    'dropout_rate': 0.2,
    'scheduler_params': {
      'factor': 0.5,
      'min_lr': 1e-6
    },
  }
  # Validation Accuracy: 48.34%
  # Test Accuracy: 37.04%

def get_training_config():
  return {
    'hidden_size': 512,
    'learning_rate': 1e-3,
    'num_epochs': 200,
    'weight_decay': 1e-4,
    'dropout_rate': 0.3,
    'scheduler_params': {
      'factor': 0.5,
      'min_lr': 1e-6
    },
  }
  # Validation Accuracy: 50.71%
  # Test Accuracy: 40.21%

  def get_training_config():
    return {
      'hidden_size': 512,
      'learning_rate': 1e-4,
      'num_epochs': 200,
      'weight_decay': 1e-4,
      'dropout_rate': 0.2,
      'scheduler_params': {
        'factor': 0.5,
        'min_lr': 1e-6
      },
    }
    # with num_layers = 2, self.input_dropout = nn.Dropout(0.2)
    # Validation Accuracy: 53.08%
    # Test Accuracy: 37.04%

  def get_training_config():
    return {
      'hidden_size': 512,
      'learning_rate': 1e-4,
      'num_epochs': 200,
      'weight_decay': 1e-4,
      'dropout_rate': 0.3,
      'scheduler_params': {
        'factor': 0.5,
        'min_lr': 1e-6
      },
    }
    # with num_layers = 2, self.input_dropout = nn.Dropout(0.2)
    # Validation Accuracy: 54.03%
    # Test Accuracy: 41.27%

  def get_training_config():
    return {
      'hidden_size': 512,
      'learning_rate': 1e-4,
      'num_epochs': 200,
      'weight_decay': 1e-4,
      'dropout_rate': 0.3,
      'scheduler_params': {
        'factor': 0.5,
        'min_lr': 1e-6
      },
    }
    # with num_layers = 2
    # Validation Accuracy: 49.76%
    # Test Accuracy: 40.21%

  def get_training_config():
    return {
      'hidden_size': 512,
      'learning_rate': 1e-4,
      'num_epochs': 200,
      'weight_decay': 1e-5,
      'dropout_rate': 0.3,
      'scheduler_params': {
        'factor': 0.5,
        'min_lr': 1e-6
      },
    }
    # with num_layers = 2
    # Validation Accuracy: 49.29%
    # Test Accuracy: 35.45%

def get_training_config():
  return {
    'hidden_size': 512,
    'learning_rate': 1e-4,
    'num_epochs': 200,
    'weight_decay': 1e-4,
    'dropout_rate': 0.2,
    'scheduler_params': {
      'factor': 0.5,
      'min_lr': 1e-6
    },
  }
    # with num_layers = 2
    # Validation Accuracy: 47.39%
    # Test Accuracy: 38.10%

Best params withoug visibility information (GRU)

In [ ]:
  def get_training_config():
    return {
      'hidden_size': 512,
      'learning_rate': 1e-3,
      'num_epochs': 100,
      'weight_decay': 1e-4,
      'dropout_rate': 0.2,
      'scheduler_params': {
        'factor': 0.5,
        'min_lr': 1e-6
      },
    }
    # Validation Accuracy: 31.75%
    # Test Accuracy: 22.75%


  def get_training_config():
    return {
      'hidden_size': 256,
      'learning_rate': 1e-3,
      'num_epochs': 100,
      'weight_decay': 1e-4,
      'dropout_rate': 0.2,
      'scheduler_params': {
        'factor': 0.5,
        'min_lr': 1e-6
      },
    }
    # Validation Accuracy: 42.18%
    # Test Accuracy: 28.57%

  def get_training_config():
    return {
      'hidden_size': 128,
      'learning_rate': 1e-3,
      'num_epochs': 100,
      'weight_decay': 1e-4,
      'dropout_rate': 0.3,
      'scheduler_params': {
        'factor': 0.5,
        'min_lr': 1e-6
      },
    }
    # Validation Accuracy: 41.23%
    # Test Accuracy: 37.04%

  def get_training_config():
    return {
      'hidden_size': 128,
      'learning_rate': 1e-3,
      'num_epochs': 100,
      'weight_decay': 1e-4,
      'dropout_rate': 0.3,
      'scheduler_params': {
        'factor': 0.5,
        'min_lr': 1e-6
      },
    }
    # with num_layers = 2, self.input_dropout = nn.Dropout(0.2)
    # Validation Accuracy: 48.82%
    # Test Accuracy: 37.04%

  def get_training_config():
    return {
      'hidden_size': 128,
      'learning_rate': 1e-3,
      'num_epochs': 100,
      'weight_decay': 1e-4,
      'dropout_rate': 0.4,
      'scheduler_params': {
        'factor': 0.5,
        'min_lr': 1e-6
      },
    }
    # with num_layers = 2, self.input_dropout = nn.Dropout(0.2)
    # Validation Accuracy: 46.92%
    # Test Accuracy: 34.92%

  def get_training_config():
    return {
      'hidden_size': 128,
      'learning_rate': 1e-3,
      'num_epochs': 100,
      'weight_decay': 1e-4,
      'dropout_rate': 0.3,
      'scheduler_params': {
        'factor': 0.5,
        'min_lr': 1e-6
      },
    }
    # with num_layers = 2
    # Validation Accuracy: 49.76%
    # Test Accuracy: 40.74%

  def get_training_config():
    return {
      'hidden_size': 128,
      'learning_rate': 1e-3,
      'num_epochs': 100,
      'weight_decay': 1e-4,
      'dropout_rate': 0.4,
      'scheduler_params': {
        'factor': 0.5,
        'min_lr': 1e-6
      },
    }
    # with num_layers = 2
    # Validation Accuracy: 51.18%
    # Test Accuracy: 37.57%

  def get_training_config():
    return {
      'hidden_size': 256,
      'learning_rate': 1e-3,
      'num_epochs': 100,
      'weight_decay': 1e-4,
      'dropout_rate': 0.2,
      'scheduler_params': {
        'factor': 0.5,
        'min_lr': 1e-6
      },
    }
    # with num_layers = 2, self.input_dropout = nn.Dropout(0.2)
    # Validation Accuracy: 39.81%
    # Test Accuracy: 33.86%

  def get_training_config():
    return {
      'hidden_size': 256,
      'learning_rate': 1e-3,
      'num_epochs': 100,
      'weight_decay': 1e-4,
      'dropout_rate': 0.2,
      'scheduler_params': {
        'factor': 0.5,
        'min_lr': 1e-6
      },
    }
    # with num_layers = 2
    # Validation Accuracy: 46.45%
    # Test Accuracy: 32.80%